교재 예제처럼 spark.read로 csv를 불러오려다 무한 오류에 빠졌었습니다. (ec2)

그래서 pandas로 받아와서 DataFrame으로 변환하게 되었습니다.

***수정(2021.08.17)***
데이터 파일을 하둡 디렉토리로 가져와야 하는데 해당 ipynb 파일의 경로로 가져와 오류가 발생했습니다.
하둡 경로에 파일을 업로드 하려면
hdfs dfs -put [파일 원경로] /user/ubuntu/...
을 사용하여 가져와야 합니다.

또한, SQL을 사용할 때에는 createOrReplaceTempView를 이전에 꼭 적용해야 합니다.

In [ ]:
# ms 제공 예제 : pd -> df
# Generate a pandas DataFrame
pdf = pd.DataFrame(np.random.rand(100, 3))

# Create a Spark DataFrame from a pandas DataFrame using Arrow
df = spark.createDataFrame(pdf)

In [5]:
spark

In [2]:
myRange = spark.range(1000).toDF('number')
# 1개의 컬럼과 1000개의 로우. 각 로우에 0~999까지의 값이 할당
# 각 부분이 서로 다른 익스큐터에 할당됨

In [3]:
divisBy2 = myRange.where('number % 2 = 0')
# 액션을 하지 않아 결과 출력 x
# 짝수 찾는 예제

In [4]:
# 액션
divisBy2.count()

500

In [1]:
pwd

'/home/ubuntu/hadoop-2.9.0/spark-defintive-guide'

In [2]:
ls ./flight-data

2010-summary.csv  2012-summary.csv  2014-summary.csv
2011-summary.csv  2013-summary.csv  2015-summary.csv


In [1]:
# 21.08.17 사용하지 않아도 됨
pip install pyarrow

     |████████████████████████████████| 22.6MB 4.3MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [1]:
# 21.08.17 사용하지 않아도 됨
import pyarrow
import pandas as pd
from pyarrow import csv

In [1]:
flightData2015 = spark\
.read\
.option('inferSchema', 'true')\
.option('header', 'true')\
.csv('./ybigta_de/input/2015-summary.csv')

In [4]:
# 21.08.17 사용하지 않아도 됨
flightData2015pandas = pd.read_csv('./flight-data/2015-summary.csv')
flightData2015 = spark.createDataFrame(flightData2015pandas)

In [3]:
print(flightData2015)

DataFrame[DEST_COUNTRY_NAME: string, ORIGIN_COUNTRY_NAME: string, count: int]


In [4]:
# head와 동일
flightData2015.take(3)

[Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Romania', count=15),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Croatia', count=1),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Ireland', count=344)]

In [5]:
flightData2015.sort('count').explain()

== Physical Plan ==
*(2) Sort [count#12 ASC NULLS FIRST], true, 0
+- Exchange rangepartitioning(count#12 ASC NULLS FIRST, 200)
   +- *(1) FileScan csv [DEST_COUNTRY_NAME#10,ORIGIN_COUNTRY_NAME#11,count#12] Batched: false, Format: CSV, Location: InMemoryFileIndex[hdfs://localhost:9010/user/ubuntu/ybigta_de/input/2015-summary.csv], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<DEST_COUNTRY_NAME:string,ORIGIN_COUNTRY_NAME:string,count:int>


In [6]:
spark.conf.set('spark.sql.shuffle.partitions', '5')
flightData2015.sort('count').take(2)

[Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Singapore', count=1),
 Row(DEST_COUNTRY_NAME='Moldova', ORIGIN_COUNTRY_NAME='United States', count=1)]

In [7]:
# 모든 df를 테이블이나 뷰로 만듦
# SQL 사용 가능
flightData2015.createOrReplaceTempView("flight_data_2015")

In [11]:
# 두 가지 모두 동일한 실행 계획으로 컴파일 됨

sqlWay = spark.sql("""
SELECT DEST_COUNTRY_NAME, count(1)
FROM flight_data_2015
GROUP BY DEST_COUNTRY_NAME
""")

dataFrameWay = flightData2015\
.groupBy("DEST_COUNTRY_NAME")\
.count()

# sqlWay.explain()
dataFrameWay.explain()

== Physical Plan ==
*(2) HashAggregate(keys=[DEST_COUNTRY_NAME#10], functions=[count(1)])
+- Exchange hashpartitioning(DEST_COUNTRY_NAME#10, 200)
   +- *(1) HashAggregate(keys=[DEST_COUNTRY_NAME#10], functions=[partial_count(1)])
      +- *(1) FileScan csv [DEST_COUNTRY_NAME#10] Batched: false, Format: CSV, Location: InMemoryFileIndex[hdfs://localhost:9010/user/ubuntu/ybigta_de/input/2015-summary.csv], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<DEST_COUNTRY_NAME:string>


In [13]:
from pyspark.sql.functions import max

flightData2015.select(max('count')).take(1)

[Row(max(count)=370002)]

In [15]:
# 둘 모두 동일한 결과

maxSql = spark.sql("""
SELECT DEST_COUNTRY_NAME, sum(count) as destination_total
FROM flight_data_2015
GROUP BY DEST_COUNTRY_NAME
ORDER BY sum(count) DESC
LIMIT 5
""")
maxSql.show()

from pyspark.sql.functions import desc
flightData2015\
.groupBy('DEST_COUNTRY_NAME')\
.sum('count')\
.withColumnRenamed('sum(count)', 'destination_total')\
.sort(desc('destination_total'))\
.limit(5)\
.show()

+-----------------+-----------------+
|DEST_COUNTRY_NAME|destination_total|
+-----------------+-----------------+
|    United States|           411352|
|           Canada|             8399|
|           Mexico|             7140|
|   United Kingdom|             2025|
|            Japan|             1548|
+-----------------+-----------------+

+-----------------+-----------------+
|DEST_COUNTRY_NAME|destination_total|
+-----------------+-----------------+
|    United States|           411352|
|           Canada|             8399|
|           Mexico|             7140|
|   United Kingdom|             2025|
|            Japan|             1548|
+-----------------+-----------------+



In [28]:
flightData2015\
.groupBy('DEST_COUNTRY_NAME')\
.sum('count')\
.withColumnRenamed('sum(count)', 'destination_total')\
.sort(desc('destination_total'))\
.limit(5)\
.explain()

== Physical Plan ==
TakeOrderedAndProject(limit=5, orderBy=[destination_total#68L DESC NULLS LAST], output=[DEST_COUNTRY_NAME#12,destination_total#68L])
+- *(2) HashAggregate(keys=[DEST_COUNTRY_NAME#12], functions=[sum(count#14L)])
   +- Exchange hashpartitioning(DEST_COUNTRY_NAME#12, 5)
      +- *(1) HashAggregate(keys=[DEST_COUNTRY_NAME#12], functions=[partial_sum(count#14L)])
         +- *(1) Project [DEST_COUNTRY_NAME#12, count#14L]
            +- Scan ExistingRDD[DEST_COUNTRY_NAME#12,ORIGIN_COUNTRY_NAME#13,count#14L]
